### Note: requires ganache-cli to be installed
### Start ganache-cli as ganache-cli -d --db /data2/web3/ganachedb -i 1234

In [1]:
from web3 import Web3

# Code to compile .sol file and export EVM bytecode and ABI to JSON

In [2]:
contract_path = "RepositoryTracker.sol"
with open(contract_path, "r") as file:
    contract_file = file.read()
print(contract_file)

// SPDX-License-Identifier: MIT
pragma solidity >=0.8.17;

contract RepositoryTracker {
    string private _ipfsAddress;
    string private _friendlyName;
    address private _owner;

    constructor(string memory bafyHash, string memory friendlyName) {
        _owner = msg.sender;
        _ipfsAddress = bafyHash;
        _friendlyName = friendlyName;
    }

// Ownership stuff
    function owner() public view virtual returns (address) {
        return _owner;
    }
    
    modifier onlyOwner() {
        require(
            owner() == msg.sender,
            "Ownership Assertion: Caller of the function is not the owner."
        );
        _;
    }

    function transferOwnership(address newOwner) public virtual onlyOwner {
        _owner = newOwner;
    }
// End ownership stuff!
// TODO: extend ownership to contributors or other relations
// TODO: migrate to OpenZeppelin Ownable class

    function setIpfsAddress(string calldata bafyHash) public onlyOwner {
        _ipfsAddress = baf

In [3]:
from solcx import compile_standard

compiled_solidity = compile_standard(
    {
        "language": "Solidity",
        "sources": {
            "RepositoryTracker.sol": {
                "content": contract_file,
            }
        },
        "settings": {
            "outputSelection": {
                "*": {"*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]}
            }
        },
    },
    solc_version="0.8.17",
)

In [4]:
import json

with open("compiled_contract.json", "w") as file:
    json.dump(compiled_solidity, file)

# Get ABI and EVM bytecode

In [5]:
abi = compiled_solidity['contracts']['RepositoryTracker.sol']['RepositoryTracker']['abi']
abi

[{'inputs': [{'internalType': 'string', 'name': 'bafyHash', 'type': 'string'},
   {'internalType': 'string', 'name': 'friendlyName', 'type': 'string'}],
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'inputs': [],
  'name': 'getFriendlyName',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'getIpfsAddress',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'owner',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'string',
    'name': 'friendlyName',
    'type': 'string'}],
  'name': 'setFriendlyName',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'string', 'name': 'bafyHash', 'type': 'string'}],
  'name':

In [6]:
bytecode = compiled_solidity['contracts']['RepositoryTracker.sol']['RepositoryTracker']['evm']['bytecode']['object']
# bytecode

# Connect to Test net (Goerli/Ganache local)

In [7]:
ropsten_url = 'https://goerli.infura.io/v3/5953d19885cd43218c5d03280949a537'
ganache_url = 'http://localhost:8545'

w3 = Web3(Web3.HTTPProvider(ganache_url))
w3.isConnected()

True

In [8]:
RepositoryTracker = w3.eth.contract(abi=abi, bytecode=bytecode)
RepositoryTracker

web3._utils.datatypes.Contract

# Deploy Transaction (each repository to be tracked requires a new contract to be deployed)

In [9]:
# attributes to build transaction
goerli_chain = 5
ganache_chain = 1337

metamask_wallet = '0x47f9aEB8F599d320718C92e30C406A893638253E'
metamask_wallet_private_key = ''

ganache_wallet = '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'
ganache_wallet_private_key = '0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d'


# change settings here
chainId = ganache_chain
wallet = ganache_wallet
wallet_private_key = ganache_wallet_private_key

nonce = w3.eth.getTransactionCount(wallet)
nonce

24

## Build deployment transaction

In [10]:
transaction = RepositoryTracker.constructor('hash', 'reponame').buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": wallet,
        "chainId": chainId,
        "nonce": nonce,
    }
)

## Sign transaction with private key

In [11]:
signed_transaction = w3.eth.account.sign_transaction(transaction, private_key=wallet_private_key)
# signed_transaction

## Perform transaction

In [12]:
tx_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# update nonce after each successful transaction: each transaction needs to have a unique nonce
nonce += 1

tx_hash

HexBytes('0xf4fdd0683a82a0a0170a908dd416f2a2383112431b56d0b80eb08af13582a913')

In [13]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xf4fdd0683a82a0a0170a908dd416f2a2383112431b56d0b80eb08af13582a913'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x3e3be81a52ef6111369b8a6dd35988a65630733b5295513e8cae80527346be27'),
 'blockNumber': 25,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'gasUsed': 705005,
 'cumulativeGasUsed': 705005,
 'contractAddress': '0xD86C8F0327494034F60e25074420BcCF560D5610',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

## Capture contract address

In [14]:
contract_address = tx_receipt.contractAddress
contract_address

'0xD86C8F0327494034F60e25074420BcCF560D5610'

# Get Contract Instance from Test net

In [15]:
contract_instance = w3.eth.contract(address=contract_address, abi=abi)
contract_instance

## Getter call

In [16]:
contract_instance.functions.getIpfsAddress().call()

'hash'

## Simulate setter transaction

In [17]:
contract_instance.functions.setIpfsAddress('settingBafyAddr').call()

[]

## Setter call

In [18]:
set_ipfs_addr_transaction = contract_instance.functions.setIpfsAddress('fakeBafyHash').buildTransaction({
        "gasPrice": w3.eth.gas_price,
        "from": wallet,
        "chainId": chainId,
        "nonce": nonce,
})

In [19]:
signed_set_ipfs_addr_transaction = w3.eth.account.sign_transaction(set_ipfs_addr_transaction, private_key=wallet_private_key)

In [20]:
set_ipfs_tx_hash = w3.eth.send_raw_transaction(signed_set_ipfs_addr_transaction.rawTransaction)
set_ipfs_tx_receipt = w3.eth.wait_for_transaction_receipt(set_ipfs_tx_hash)

# update nonce after each successful transaction: each transaction needs to have a unique nonce
nonce += 1

set_ipfs_tx_receipt

AttributeDict({'transactionHash': HexBytes('0xfb0f9fa1cede6f2b0ebb4d7ceb49cae01c5e2f41b71f4659d057360024b7e92f'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x30b08cd508164da8e10903fc42d039116bfb802bb24eb7aa9f475213be1fc7d8'),
 'blockNumber': 26,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xD86C8F0327494034F60e25074420BcCF560D5610',
 'gasUsed': 29417,
 'cumulativeGasUsed': 29417,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [21]:
contract_instance.functions.getIpfsAddress().call()

'fakeBafyHash'

## Test bafyHash with non-owner

In [22]:
try:
    set_ipfs_addr_transaction = contract_instance.functions.setIpfsAddress('fakeBafyHash').buildTransaction({
        "gasPrice": w3.eth.gas_price,
        "from": '0x016Cf2768A5061AeD1e47DF56E9d09Acfe184692',
        "chainId": chainId,
        "nonce": nonce,
    })
    signed_set_ipfs_addr_transaction = w3.eth.account.sign_transaction(set_ipfs_addr_transaction, private_key='0x0f14c58877d8dbc422f158fbf1b654f8068c4f7cdbb8cdbcae2e671c78969713')

    set_ipfs_tx_hash = w3.eth.send_raw_transaction(signed_set_ipfs_addr_transaction.rawTransaction)
    set_ipfs_tx_receipt = w3.eth.wait_for_transaction_receipt(set_ipfs_tx_hash)

    # update nonce after each successful transaction: each transaction needs to have a unique nonce
    nonce += 1

    set_ipfs_tx_receipt
except Exception as e:
    print(type(e))
    # print(e.with_traceback())

<class 'web3.exceptions.ContractLogicError'>


ContractLogicError: execution reverted: VM Exception while processing transaction: revert Ownership Assertion: Caller of the function is not the owner.

# L1 Integration with dGit

## dGit init

In [23]:
! pwd

/data2/web3


In [24]:
#  creating a test repository
%cd /data2/web3/
! rm -rf test-repo
! mkdir test-repo

/data2/web3


In [25]:
# marking dgit as executable
! chmod +x dgit.sh

In [26]:
# initialize git repository, stage changes, and commit
%cd test-repo
! pwd

! ../dgit.sh -c init

! echo "hello world 2" > test.txt
! cat test.txt
! git add .
! git commit -m "test test"

/data2/web3/test-repo
/data2/web3/test-repo
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
init
Using default password! 

Empty directory detected... creating README.md
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /data2/web3/test-repo/.git/
[master (root-commit) 7c47aee] Initialized new repository
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 README

In [27]:
# capture bafyHash and friendlyName from .git/description

with open(".git/description", "r") as file:
    bafyHash = file.readline().strip()
    friendlyName = file.readline().strip()

bafyHash, friendlyName

('Qmee8P1DWqeh7ZHy1J1RvsAK55EZv1duT31XPkaH9nLZwz', 'test-repo')

### Create contract with repo info

In [28]:
new_repo_tx = RepositoryTracker.constructor(bafyHash, friendlyName).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": wallet,
        "chainId": chainId,
        "nonce": nonce,
    }
)

new_repo_tx_signed = w3.eth.account.sign_transaction(new_repo_tx, private_key=wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(new_repo_tx_signed.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

nonce += 1

contract_address = tx_receipt.contractAddress

tx_receipt

AttributeDict({'transactionHash': HexBytes('0x7b0fef9cc99ec74e6da2f506e6dc70510fb9dbcce52d347d194d8e6b067900f2'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x90a6816a045099d72c359bee9db1c87954f3e457dae45740431086a831b8fc24'),
 'blockNumber': 27,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': None,
 'gasUsed': 745915,
 'cumulativeGasUsed': 745915,
 'contractAddress': '0xaD888d0Ade988EbEe74B8D4F39BF29a8d0fe8A8D',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

### Read contract and verify changes

In [29]:
contract_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
display(contract_instance.functions.getIpfsAddress().call())
display(contract_instance.functions.getFriendlyName().call())

'Qmee8P1DWqeh7ZHy1J1RvsAK55EZv1duT31XPkaH9nLZwz'

'test-repo'

## dGit push

In [30]:
! touch newfile
! git add .
! git commit -m "test push"
! ../dgit.sh -c push -k "password"

[master 0f02610] test push
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 newfile
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
push password
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/Qmee8P1DWqeh7ZHy1J1RvsAK55EZv1duT31XPkaH9nLZwz.enc
 17.46 KiB / 17.46 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Pushing repository... 

Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 473 bytes | 473.00 KiB/s, done.
Total 5 (delta 1), reused 0 (delta 0), pack-reused 0
To /home/rahul/.tmp/bare/test-repo
   7c47aee..0f02610  master -> master
 20.15 KiB / 20.15 KiB [=========================================

### Update bafyHash (and friendlyName) on repo contract

In [31]:
# capture bafyHash and friendlyName from .git/description

with open(".git/description", "r") as file:
    bafyHash = file.readline().strip()
    friendlyName = file.readline().strip()

display(bafyHash, friendlyName)

contract_instance = w3.eth.contract(address=contract_address, abi=abi)

update_repo_contract_tx = contract_instance.functions.setIpfsAddress(bafyHash).buildTransaction(
    {
        "gasPrice": w3.eth.gas_price,
        "from": wallet,
        "chainId": chainId,
        "nonce": nonce,
    }
)

update_repo_contract_tx_signed = w3.eth.account.sign_transaction(update_repo_contract_tx, private_key=wallet_private_key)
tx_hash = w3.eth.send_raw_transaction(update_repo_contract_tx_signed.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

nonce += 1

tx_receipt

'QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh'

'test-repo'

AttributeDict({'transactionHash': HexBytes('0xfaeef4803245f55623d9eaae3c043c8fb4cf1456641fed0d9c9a905239da7f65'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x93b13510e0d9d284f26bf453a40bdbb17ca7defe8c76cf580451dea4abd8e387'),
 'blockNumber': 28,
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0xaD888d0Ade988EbEe74B8D4F39BF29a8d0fe8A8D',
 'gasUsed': 36249,
 'cumulativeGasUsed': 36249,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [32]:
# verifying changes
# contract_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
display(contract_instance.functions.getIpfsAddress().call())
display(contract_instance.functions.getFriendlyName().call())

'QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh'

'test-repo'

## dGit clone

In [35]:
%cd /data2/web3/
! pwd
! rm -rf test-repo/
! ./dgit.sh -c clone -k "password" -i QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh

/data2/web3
/data2/web3
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
clone password QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh.enc
 20.15 KiB / 20.15 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Cloning repository... 

Cloning into 'test-repo'...
done.

Cleaning up... 

Writing repository tracking information to disk... 



## dGit pull

In [36]:
%cd /data2/web3/test-repo/
! pwd
! ../dgit.sh -c pull -k password -i QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh

/data2/web3/test-repo
/data2/web3/test-repo
--------------------------------------------------------------------------------
Decentralized-Git-on-IPFS Wrapper
--------------------------------------------------------------------------------
pull password QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh
Fetching repository from IPFS... 

Saving file(s) to /home/rahul/.tmp/QmWQFRsYCegGEuaZMDJgZKWU3KpmATXoctE8Qwrsg76vgh.enc
 20.15 KiB / 20.15 KiB [============================================] 100.00% 0s

Decrypting contents... 

Friendly repository name: test-repo

Pulling repository... 

From /home/rahul/.tmp/bare/test-repo
 * branch            HEAD       -> FETCH_HEAD
Already up to date.

Cleaning up... 



### TODO: Extend Ownable to implement contributors
### TODO: Implement code as .py and integrate with dGit